# WEEK 37


In [21]:
import math
from collections import Counter
from typing import List, Dict, Iterable

import pandas as pd

# NLTK
from nltk.tokenize import wordpunct_tokenize
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.lm import MLE, Laplace, KneserNeyInterpolated,  WittenBellInterpolated


In [22]:
## K = 3

#DOWNLOAD DATASET

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df_train = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])
print(df_train.head())
print(df_val.head())

                                            question  \
0  উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...   
1           দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?   
2  মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...   
3  আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...   
4          বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?   

                                             context lang  answerable  \
0  WikiLeaks () is an international non-profit or...   bn        True   
1  The war in Europe concluded with an invasion o...   bn        True   
2  Same-sex marriage in the United States expande...   bn       False   
3  The exact number of Arab casualties is unknown...   bn        True   
4  As Thomas Hall (2000) notes, "The Sung Empire ...   bn        True   

   answer_start        answer answer_inlang  
0           182          2006          None  
1            48       Germany          None  
2            -1            no          None  
3            39       unknown          N

In [23]:

QUESTION_FIELD = "question"

CONTEXT_FIELD = "context"

LANG_FIELD = "lang"

LANGUAGES = ["ar", "ko", "te", "en"]

N_ORDERS = [2, 3]

MIN_FREQ = 3

SMOOTHINGS= ["laplace", "kn", "wb"]


In [24]:
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # ne
from collections import Counter
from typing import List, Dict

from camel_tools.tokenizers.word import simple_word_tokenize      # Arabic
from konlpy.tag import Okt                                        # Korean
from indicnlp.tokenize import indic_tokenize                      # Telugu

okt = Okt()

def get_tokenizer(lang: str):
    if lang == "en":
        return nltk.word_tokenize
    elif lang == "ar":
        return simple_word_tokenize
    elif lang == "ko":
        return okt.morphs
    elif lang == "te":
        return lambda text: list(indic_tokenize.trivial_tokenize(text))
    else:
        # fallback: split on words
        return nltk.word_tokenize

def tokenize_column(series, lang: str):
    tokenizer = get_tokenizer(lang)
    out = []
    for x in series:
        if not isinstance(x, str):
            x = "" if x is None else str(x)
        x = x.lower()
        out.append(tokenizer(x))
    return out

def build_vocab(tokenized: List[List[str]], min_freq: int = 1) -> Dict[str, int]:
    cnt = Counter(w for s in tokenized for w in s)
    vocab = {w: i for i, (w, c) in enumerate(cnt.items()) if c >= min_freq}
    vocab["<OOV>"] = len(vocab)
    return vocab

def apply_oov(tokenized: List[List[str]], vocab: Dict[str, int]) -> List[List[str]]:
    return [[w if w in vocab else "<OOV>" for w in s] for s in tokenized]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [25]:
def train_lm(tokenized: List[List[str]], order: int = 2, smoothing: str = "laplace"):
    smoothing = (smoothing or "").lower()
    if smoothing in {"", "mle"}:
        model = MLE(order)
    elif smoothing in {"laplace", "addone", "add-one"}:
        model = Laplace(order)
    elif smoothing in {"kn", "kneserney", "kneser-ney"}:
        model = KneserNeyInterpolated(order)
    elif smoothing in {"wb"}:
        model = WittenBellInterpolated(order)
    else:
        raise ValueError(f"Unsupported smoothing: {smoothing}")

    train_data, vocab = padded_everygram_pipeline(order, tokenized)

    model.fit(train_data, vocab)
    return model


In [26]:
def corpus_perplexity(model, tokenized: List[List[str]], order: int) -> dict:
    total_logp = 0.0
    total_count = 0
    for sent in tokenized:
        padded = list(pad_both_ends(sent, n=order))  # add <s>, </s>
        for ng in ngrams(padded, order):
            ctx = ng[:-1]
            w   = ng[-1]
            p = model.score(w, ctx)
            if p <= 0.0:
                p = 1e-12  # guard
            total_logp += math.log(p)
            total_count += 1
    ce  = - total_logp / max(total_count, 1)   # cross-entropy
    ppl = math.exp(ce)                         # perplexity
    return {"cross_entropy": ce, "perplexity": ppl, "tokens_counted": total_count}


In [27]:
results = []

for lang in LANGUAGES:
    # pick the text column(s)
    if lang == "en":
        tr_series = df_train[CONTEXT_FIELD].dropna().astype(str)
        va_series = df_val[CONTEXT_FIELD].dropna().astype(str)
        # exclude KN for English
        smoothings_here = [s for s in SMOOTHINGS
                           if s.lower() not in {"kn"}]
    else:
        tr_series = df_train[df_train[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str)
        va_series = df_val[df_val[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str)
        # use all listed smoothings
        smoothings_here = SMOOTHINGS[:]

    print(f"\n▶ Language: {lang} | train={len(tr_series)} | valid={len(va_series)}")

    # tokenize
    tr_tok = tokenize_column(tr_series, lang)
    va_tok = tokenize_column(va_series, lang)

    # vocab + <UNK>
    vocab     = build_vocab(tr_tok, min_freq=MIN_FREQ)
    tr_tok_u  = apply_oov(tr_tok, vocab)
    va_tok_u  = apply_oov(va_tok, vocab)

    # train/eval for each n and smoothing
    for n in N_ORDERS:
        for s in smoothings_here:
            lm = train_lm(tr_tok_u, order=n, smoothing=s)
            metrics = corpus_perplexity(lm, va_tok_u, order=n)
            results.append({
                "language": lang,
                "vocab_size" : len(vocab),
                "order": n,
                "smoothing": s,
                **metrics
            })
            print(f"  n={n}  {s}: PPL={metrics['perplexity']:.2f}  CE={metrics['cross_entropy']:.4f}  tokens={metrics['tokens_counted']}")



▶ Language: ar | train=2558 | valid=415
  n=2  laplace: PPL=34.56  CE=3.5428  tokens=3457
  n=2  kn: PPL=13.23  CE=2.5825  tokens=3457
  n=2  wb: PPL=12.83  CE=2.5519  tokens=3457
  n=3  laplace: PPL=54.72  CE=4.0023  tokens=3872
  n=3  kn: PPL=12.03  CE=2.4878  tokens=3872
  n=3  wb: PPL=9.78  CE=2.2801  tokens=3872

▶ Language: ko | train=2422 | valid=356
  n=2  laplace: PPL=30.41  CE=3.4148  tokens=3608
  n=2  kn: PPL=11.89  CE=2.4756  tokens=3608
  n=2  wb: PPL=11.51  CE=2.4429  tokens=3608
  n=3  laplace: PPL=49.59  CE=3.9037  tokens=3964
  n=3  kn: PPL=9.71  CE=2.2728  tokens=3964
  n=3  wb: PPL=8.16  CE=2.0992  tokens=3964

▶ Language: te | train=1355 | valid=384
  n=2  laplace: PPL=30.61  CE=3.4214  tokens=3074
  n=2  kn: PPL=10.99  CE=2.3966  tokens=3074
  n=2  wb: PPL=10.89  CE=2.3879  tokens=3074
  n=3  laplace: PPL=43.59  CE=3.7747  tokens=3458
  n=3  kn: PPL=8.54  CE=2.1443  tokens=3458
  n=3  wb: PPL=7.28  CE=1.9854  tokens=3458

▶ Language: en | train=15343 | valid=3011

In [28]:
res_df = pd.DataFrame(results).sort_values(["language","order"])
res_df


,language,vocab_size,order,smoothing,cross_entropy,perplexity,tokens_counted
0,ar,965,2,laplace,3.542808,34.563823,3457
1,ar,965,2,kn,2.582526,13.230522,3457
2,ar,965,2,wb,2.551880,12.831205,3457
3,ar,965,3,laplace,4.002280,54.722794,3872
4,ar,965,3,kn,2.487780,12.034535,3872
5,ar,965,3,wb,2.280074,9.777403,3872
18,en,29119,2,laplace,7.319152,1508.923553,353455
19,en,29119,2,wb,5.346028,209.773333,353455
20,en,29119,3,laplace,8.995849,8069.521425,356466
21,en,29119,3,wb,4.988482,146.713584,356466


### POTENTIAL NEURAL MODEL ???

In [29]:
# If needed (Colab/new env), uncomment:
# !pip install -q bpemb torch

import math, re, random
from typing import List, Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from bpemb import BPEmb


ModuleNotFoundError: No module named 'bpemb'

In [ ]:
SEED      = 42
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"

# training knobs (start small, bump after it works)
EPOCHS    = 10
BATCH     = 128
SEQ_LEN   = 120
EMB_DIM   = 100      # must match bpemb dim
HIDDEN    = 256
LAYERS    = 2
DROPOUT   = 0.2
LR        = 2e-3
CLIP      = 1.0

# bpemb options
# Option A (per-language): ar/ko/te/en separately — best quality
BPE_PER_LANGUAGE = True
BP_VS   = 5000    # vocab size
BP_DIM  = EMB_DIM   # should equal EMB_DIM

# Option B (single multilingual model): use 'multi'
# BPE_PER_LANGUAGE = False
# BP_LANG = "multi"     # uncomment to try multilingual model

# light text normalization
LOWERCASE    = True
FOLD_NUMBERS = True

# throttle English contexts (they’re huge)
EN_MAX_ROWS_TRAIN    = 30000
EN_MAX_ROWS_VALID    = 5000
EN_MAX_TOK_PER_DOC   = 400      # subword pieces per doc (for speed)
FREEZE_EMBEDDINGS    = True     # False = fine-tune embeddings


In [ ]:
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    if LOWERCASE:
        s = s.lower()
    if FOLD_NUMBERS:
        s = re.sub(r"\d+", "<num>", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def get_series(lang):
    if lang == "en":
        tr = df_train[CONTEXT_FIELD].dropna().astype(str)
        va = df_val[CONTEXT_FIELD].dropna().astype(str)
        if len(tr) > EN_MAX_ROWS_TRAIN:
            tr = tr.sample(EN_MAX_ROWS_TRAIN, random_state=SEED)
        if len(va) > EN_MAX_ROWS_VALID:
            va = va.sample(EN_MAX_ROWS_VALID, random_state=SEED)
        tr = tr.apply(normalize_text)
        va = va.apply(normalize_text)
        return tr, va, EN_MAX_TOK_PER_DOC
    else:
        tr = df_train[df_train[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str).apply(normalize_text)
        va = df_val[df_val[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str).apply(normalize_text)
        return tr, va, None


In [ ]:
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    if LOWERCASE:
        s = s.lower()
    if FOLD_NUMBERS:
        s = re.sub(r"\d+", "<num>", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def get_series(lang):
    if lang == "en":
        tr = df_train[CONTEXT_FIELD].dropna().astype(str)
        va = df_val[CONTEXT_FIELD].dropna().astype(str)
        if len(tr) > EN_MAX_ROWS_TRAIN:
            tr = tr.sample(EN_MAX_ROWS_TRAIN, random_state=SEED)
        if len(va) > EN_MAX_ROWS_VALID:
            va = va.sample(EN_MAX_ROWS_VALID, random_state=SEED)
        tr = tr.apply(normalize_text)
        va = va.apply(normalize_text)
        return tr, va, EN_MAX_TOK_PER_DOC
    else:
        tr = df_train[df_train[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str).apply(normalize_text)
        va = df_val[df_val[LANG_FIELD] == lang][QUESTION_FIELD].dropna().astype(str).apply(normalize_text)
        return tr, va, None


In [ ]:
_bp_cache = {}

def load_bpemb_for_lang(lang: str):
    if BPE_PER_LANGUAGE:
        key = f"{lang}:{BP_VS}:{BP_DIM}"
        if key not in _bp_cache:
            _bp_cache[key] = BPEmb(lang=lang, vs=BP_VS, dim=BP_DIM)
        return _bp_cache[key]
    else:
        key = f"multi:{BP_VS}:{BP_DIM}"
        if key not in _bp_cache:
            _bp_cache[key] = BPEmb(lang="multi", vs=BP_VS, dim=BP_DIM)
        return _bp_cache[key]

def build_embedding_matrix(bp: BPEmb):
    # bp.emb.vectors shape: (vs, dim)
    pad_vec = np.zeros((1, BP_DIM), dtype=np.float32)
    eos_vec = np.zeros((1, BP_DIM), dtype=np.float32)  # simple; could be mean vector
    mat = np.concatenate([bp.emb.vectors, pad_vec, eos_vec], axis=0).astype(np.float32)
    pad_id = bp.emb.vectors.shape[0]        # PAD index
    eos_id = bp.emb.vectors.shape[0] + 1    # EOS index
    return torch.tensor(mat), pad_id, eos_id


In [ ]:
def encode_docs_to_stream(bp: BPEmb, docs: List[str], eos_id: int, max_len_per_doc: int = None):
    ids = []
    for d in docs:
        wp = bp.encode_ids(d)  # list of subword ids (0..vs-1)
        if max_len_per_doc:    # throttle long docs (esp. English)
            wp = wp[:max_len_per_doc]
        ids.extend(wp + [eos_id])
    return np.array(ids, dtype=np.int64)

class WPStreamDataset(Dataset):
    def __init__(self, id_stream: np.ndarray, seq_len: int):
        self.data = torch.tensor(id_stream, dtype=torch.long)
        self.seq_len = seq_len
        self.num_seq = max(0, (len(self.data) - 1) // seq_len)

    def __len__(self): return self.num_seq
    def __getitem__(self, i):
        s = i * self.seq_len
        x = self.data[s : s+self.seq_len]
        y = self.data[s+1 : s+1+self.seq_len]
        return x, y

def make_loaders(id_train: np.ndarray, id_valid: np.ndarray, seq_len: int, batch: int):
    tr_ds = WPStreamDataset(id_train, seq_len)
    va_ds = WPStreamDataset(id_valid, seq_len)
    tr_dl = DataLoader(tr_ds, batch_size=batch, shuffle=True,  drop_last=True)
    va_dl = DataLoader(va_ds, batch_size=batch, shuffle=False, drop_last=False)
    return tr_dl, va_dl


In [ ]:
def encode_docs_to_stream(bp: BPEmb, docs: List[str], eos_id: int, max_len_per_doc: int = None):
    ids = []
    for d in docs:
        wp = bp.encode_ids(d)  # list of subword ids (0..vs-1)
        if max_len_per_doc:    # throttle long docs (esp. English)
            wp = wp[:max_len_per_doc]
        ids.extend(wp + [eos_id])
    return np.array(ids, dtype=np.int64)

class WPStreamDataset(Dataset):
    def __init__(self, id_stream: np.ndarray, seq_len: int):
        self.data = torch.tensor(id_stream, dtype=torch.long)
        self.seq_len = seq_len
        self.num_seq = max(0, (len(self.data) - 1) // seq_len)

    def __len__(self): return self.num_seq
    def __getitem__(self, i):
        s = i * self.seq_len
        x = self.data[s : s+self.seq_len]
        y = self.data[s+1 : s+1+self.seq_len]
        return x, y

def make_loaders(id_train: np.ndarray, id_valid: np.ndarray, seq_len: int, batch: int):
    tr_ds = WPStreamDataset(id_train, seq_len)
    va_ds = WPStreamDataset(id_valid, seq_len)
    tr_dl = DataLoader(tr_ds, batch_size=batch, shuffle=True,  drop_last=True)
    va_dl = DataLoader(va_ds, batch_size=batch, shuffle=False, drop_last=False)
    return tr_dl, va_dl


In [ ]:
class WPLSTMLM(nn.Module):
    def __init__(self, emb_matrix: torch.Tensor, pad_id: int, hidden=256, layers=2, dropout=0.2, freeze=True):
        super().__init__()
        V, D = emb_matrix.shape
        self.embed = nn.Embedding.from_pretrained(emb_matrix, freeze=freeze, padding_idx=pad_id)
        self.lstm  = nn.LSTM(D, hidden, num_layers=layers, dropout=dropout, batch_first=True)
        self.drop  = nn.Dropout(dropout)
        self.head  = nn.Linear(hidden, V)

    def forward(self, x, h=None):
        x = self.embed(x)
        x, h = self.lstm(x, h)
        x = self.drop(x)
        logits = self.head(x)
        return logits, h


In [ ]:
def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_tok = 0.0, 0
    crit = nn.CrossEntropyLoss(reduction="sum")  # sum, we divide later
    for x, y in loader:
        x = x.to(DEVICE); y = y.to(DEVICE)
        logits, _ = model(x)
        loss = crit(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
        total_loss += loss.item()
        total_tok  += y.numel()
    ce  = total_loss / max(total_tok, 1)   # natural log
    ppl = math.exp(ce)
    return ce, ppl, total_tok

def train_bpemb_lm(lang, train_series, valid_series, max_tok_per_doc=None):
    set_seed()
    bp   = load_bpemb_for_lang(lang if BPE_PER_LANGUAGE else "multi")
    embM, pad_id, eos_id = build_embedding_matrix(bp)

    ids_tr = encode_docs_to_stream(bp, list(train_series), eos_id, max_tok_per_doc)
    ids_va = encode_docs_to_stream(bp, list(valid_series), eos_id, max_tok_per_doc)

    tr_dl, va_dl = make_loaders(ids_tr, ids_va, SEQ_LEN, BATCH)

    model = WPLSTMLM(embM, pad_id, hidden=HIDDEN, layers=LAYERS, dropout=DROPOUT, freeze=FREEZE_EMBEDDINGS).to(DEVICE)
    opt   = torch.optim.AdamW(model.parameters(), lr=LR)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

    print(f"[{lang}] subword_vocab={embM.size(0)} train_batches={len(tr_dl)} valid_batches={len(va_dl)} device={DEVICE}")

    best = float("inf")
    for ep in range(1, EPOCHS+1):
        model.train()
        total, steps = 0.0, 0
        crit = nn.CrossEntropyLoss()
        for x, y in tr_dl:
            x = x.to(DEVICE); y = y.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            logits, _ = model(x)
            loss = crit(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP)
            opt.step()
            total += loss.item(); steps += 1
        sched.step()
        ce, ppl, tok = evaluate(model, va_dl)
        best = min(best, ppl)
        print(f"  epoch {ep:02d}  train_ce={total/max(steps,1):.4f}  val_ce={ce:.4f}  val_ppl={ppl:.2f}  tokens={tok}")
    return best


In [ ]:
bpemb_results = []
for lang in LANGUAGES:
    tr, va, max_len = get_series(lang)
    print(f"\n▶ BPEmb LSTM on {lang}: train_rows={len(tr)} valid_rows={len(va)}")
    best_ppl = train_bpemb_lm(lang if lang!="en" else "en", tr, va, max_tok_per_doc=max_len)
    bpemb_results.append({"language": lang, "model": "bpemb_lstm", "best_ppl": best_ppl})

bpemb_results
